<a href="https://colab.research.google.com/github/chunxiaozhang2015/AdminLTE/blob/master/ANLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy[transformers,lookups]

In [ ]:
!pip install -U spacy-lookups-data
!pip install cupy-cuda110
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 6.2 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/cupy/_environment.py:447: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda110, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
/usr/local/lib/python3.10/dist-packages/cupy

In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [ ]:
sentence = """we live in China"""

In [ ]:
sentences = nlp.tokenizer(sentence)
for word in sentences:
  print(word)

we
live
in
China


In [ ]:
import pandas as pd
import os
cwd = os.getcwd()
print(cwd)

/content


In [ ]:
from typing_extensions import dataclass_transform
data = pd.read_csv(cwd + '/JEOPARDY_CSV.csv')
data = pd.DataFrame(data = data)

In [ ]:
data.columns = map(lambda x: x.lower().strip(), data.columns)

In [ ]:
data = data[0:1000]

In [ ]:
data["question_tokens"] = data["question"].apply(lambda x:nlp(x))

In [ ]:
example_question = data.question[0]
example_question_tokens = data.question_tokens[0]
print("The first question is:")
print(example_question)

The first question is:
For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory


In [ ]:
print("The tokens from the first question are:")
for tokens in example_question_tokens:
  print(tokens)

The tokens from the first question are:
For
the
last
8
years
of
his
life
,
Galileo
was
under
house
arrest
for
espousing
this
man
's
theory


In [ ]:
print("Here are the Part-of-speech tags for each token in the first question:")
for token in example_question_tokens:
  print(token.text, token.pos_, spacy.explain(token.pos_))

Here are the Part-of-speech tags for each token in the first question:
For ADP adposition
the DET determiner
last ADJ adjective
8 NUM numeral
years NOUN noun
of ADP adposition
his PRON pronoun
life NOUN noun
, PUNCT punctuation
Galileo PROPN proper noun
was AUX auxiliary
under ADP adposition
house NOUN noun
arrest NOUN noun
for ADP adposition
espousing VERB verb
this DET determiner
man NOUN noun
's PART particle
theory NOUN noun


In [ ]:
for token in example_question_tokens:
  print(token.text, token.dep_, spacy.explain(token.dep_))

For prep prepositional modifier
the det determiner
last amod adjectival modifier
8 nummod numeric modifier
years pobj object of preposition
of prep prepositional modifier
his poss possession modifier
life pobj object of preposition
, punct punctuation
Galileo nsubj nominal subject
was ROOT root
under prep prepositional modifier
house compound compound
arrest pobj object of preposition
for prep prepositional modifier
espousing pcomp complement of preposition
this det determiner
man poss possession modifier
's case case marking
theory dobj direct object


In [ ]:
from spacy import displacy
displacy.render(example_question_tokens, style='dep',jupyter=True, options={'distance': 120})

In [ ]:
for token in nlp("My parents live in New York City."):
  print(token.text)

My
parents
live
in
New
York
City
.


In [ ]:
for chunk in nlp("My parents live in New York City.").noun_chunks:
  print(chunk.text)

My parents
New York City


In [ ]:
example_sentence = """George Washington was an American political leader, military general, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797."""
print(example_sentence)
print("Text Start End Label")
doc = nlp(example_sentence)
for token in doc.ents:
  print(token.text, token.start_char, token.end_char, token.label_)
displacy.render(doc, style="ent", jupyter=True, options={'distance': 120})

George Washington was an American political leader, military general, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797.
Text Start End Label
George Washington 0 17 PERSON
American 25 33 NORP
first 119 124 ORDINAL
the United States 138 155 GPE


In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
dls = TextDataLoaders.from_folder(path, valid='test')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj titanic directed by xxmaj james xxmaj cameron presents a fictional love story on the historical setting of the xxmaj titanic . xxmaj the plot is simple , xxunk , or not for those who love plots that twist and turn and keep you in suspense . xxmaj the end of the movie can be figured out within minutes of the start of the film , but the love story is an interesting one , however . xxmaj kate xxmaj winslett is wonderful as xxmaj rose , an aristocratic young lady betrothed by xxmaj cal ( billy xxmaj zane ) . xxmaj early on the voyage xxmaj rose meets xxmaj jack ( leonardo dicaprio ) , a lower class artist on his way to xxmaj america after winning his ticket aboard xxmaj titanic in a poker game . xxmaj if he wants something , he goes and gets it",pos
2,"xxbos xxmaj this movie was recently released on xxup dvd in the xxup us and i finally got the chance to see this hard - to - find gem . xxmaj it even came with original theatrical previews of other xxmaj italian horror classics like "" xxunk "" and "" beyond xxup the xxup darkness "" . xxmaj unfortunately , the previews were the best thing about this movie . \n\n "" zombi 3 "" in a bizarre way is actually linked to the infamous xxmaj lucio xxmaj fulci "" zombie "" franchise which began in 1979 . xxmaj similarly compared to "" zombie "" , "" zombi 3 "" consists of a threadbare plot and a handful of extremely bad actors that keeps this ' horror ' trash barely afloat . xxmaj the gore is nearly non - existent ( unless one is frightened of people running around with",neg
3,"xxbos xxrep 3 * xxup spoilers xxrep 3 * xxrep 3 * xxup spoilers xxrep 3 * xxmaj continued … \n\n xxmaj from here on in the whole movie collapses in on itself . xxmaj first we meet a rogue program with the indication we 're gon na get ghosts and vampires and werewolves and the like . xxmaj we get a guy with a retarded accent talking endless garbage , two ' ghosts ' that serve no real purpose and have no character what - so - ever and a bunch of henchmen . xxmaj someone 's told me they 're vampires ( straight out of xxmaj blade 2 ) , but they 're so undefined i did n't realise . \n\n xxmaj the funny accented guy with a ridiculous name suffers the same problem as the xxmaj oracle , only for far longer and far far worse .",neg
4,"xxbos xxup warning : xxup possible xxup spoilers ( but not really - keep reading ) . a xxrep 3 h , there are so many reasons to become utterly addicted to this spoof gem that i wo n't have room to list them all . xxmaj the opening credits set the playful scene with kitsch late 1950s cartoon stills ; an enchanting xxmaj xxunk ' prez ' xxmaj xxunk mambo theme which appears to be curiously uncredited ( but his grunts are unmistakable , and no - one else did them ) ; and with familiar cast names , including xxmaj kathy xxmaj xxunk a full year before she hit with xxmaj sister xxmaj acts 1 & 2 plus xxmaj teri xxmaj hatcher from tv 's xxmaj superman . \n\n xxmaj every scene is imbued with shallow injustices flung at various actors , actresses and producers in daytime xxup",pos
5,"xxbos xxup oliver xxup twist was to have controversy as well as success following it after

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.show_results()